This jupyter notebook is used for rapid development.

In [50]:
import os
import yaml
import regex
import json




In [58]:
OPEN_API_FILE = "example_data/swagger_v2.yaml"
TAVERN_FILE_DIR = "example_data/functional"
TAVERN_FILES=[]
TAVERN_REGEX = "test_.*.tavern.yaml"
API_TARGET_URLS = ["{hsm_base_url}"] #use this to specify variables you care about!
TAVERN_CONFIG = "example_data/tavern_global_config.yaml"

# yaml.add_multi_constructor(u'!bool', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)
# yaml.add_multi_constructor(u'!anything', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)
# yaml.add_multi_constructor(u'!anybool', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)

#if there are unknown tags, ignore them, return None; I am doing this is place of using safeloader which only uses known tags
class SafeLoaderIgnoreUnknown(yaml.SafeLoader):
    def ignore_unknown(self, node):
        return None
SafeLoaderIgnoreUnknown.add_constructor(None, SafeLoaderIgnoreUnknown.ignore_unknown)

# parse all the tavern test files

test_cases = []

for file in os.listdir(TAVERN_FILE_DIR):
    file_path = os.path.join(TAVERN_FILE_DIR, file)
    if os.path.isfile(file_path):
        if regex.search(TAVERN_REGEX, file) is not None:
            test_data = {}
            test_data ["file"] = file

            with open(file_path) as stream:
                #Im not using safe load, because I have a custom constructor to turn all unknown tags into Nones
                tavern_docs = yaml.load_all(stream, Loader=SafeLoaderIgnoreUnknown)
                for doc in tavern_docs:
                    test = doc["test_name"]
                    test_data["test_name"] = test
                    for stage in doc["stages"]:
                        test_stage = test + " - " + stage["name"]
                        test_data["stage"] = stage["name"]
                        test_data["request_url"] = stage["request"]["url"]
                        test_data["request_method"] = stage["request"]["method"].upper()
                        test_cases.append(test_data)
                        #test_case = (test_stage, stage["request"]["url"], stage["request"]["method"])
                        #print(test_case)
                    # for k,v in doc.items():
                    #     print(k,v)

print(json.dumps(test_cases, indent=2))

[
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Ensure that we can call the Component Query API with node BMC xnames",
    "stage": "Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API",
    "request_url": "{hsm_base_url}/hsm/v2/State/Components/Query/{xname}",
    "request_method": "GET"
  },
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Ensure that we can call the Component Query API with node BMC xnames",
    "stage": "Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API",
    "request_url": "{hsm_base_url}/hsm/v2/State/Components/Query/{xname}",
    "request_method": "GET"
  },
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Ensure that we can call the Component Query API with node BMC xnames",
    "stage": "Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API",
    "request_url": "{hsm_base_url}/hsm/v2/St

In [59]:
# parse the swagger doc
api_methods = []
with open(OPEN_API_FILE) as stream:
    try:
        swagger = yaml.safe_load(stream)
        endpoint = {}
        for path in swagger["paths"]:
            for method in swagger["paths"][path]:
                endpoint["method"] = method.upper()
                endpoint["url"] = path
                api_methods.append(endpoint)
    except yaml.YAMLError as exc:
        print(exc)
        1/0

print(json.dumps(api_methods, indent=2))



[
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": "PUT",
    "url": "/sysinfo/powermaps/{xname}"
  },
  {
    "method": 

In [20]:
with open("configuration.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        logging.error(exc)
        exit(1)

FileNotFoundError: [Errno 2] No such file or directory: 'configuration.yaml'